<a href="https://colab.research.google.com/github/Hushpuppyzac/DLI-Assignment/blob/main/CleanedData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# STEP 1: Loading Dataset

# Use the raw URL for the CSV file on GitHub
url = "https://raw.githubusercontent.com/Hushpuppyzac/DLI-Assignment/main/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
df = pd.read_csv(url)

print("="*50)
print(" INITIAL DATASET INFO")
print("="*50)
print(f"Total rows before cleaning: {len(df)}")
# Note: ' Label' might have a leading space from the original CSV
print("Initial Label distribution:")
print(df[' Label'].value_counts())
print("\nDataFrame Info:")
df.info()
print("\nDataFrame Description (Numerical Columns):")
print(df.describe()) # Using print instead of display for consistency in non-notebook environments
print("="*50)

# ------------------------------------------------------------------------------

# STEP 2: Data Cleaning (Applied to the full dataset before splitting)

# Strip leading/trailing whitespaces in column names
df.columns = df.columns.str.strip()

# Explicitly rename the ' Label' column to 'Label' after stripping
# This handles cases where the column name might have had a leading space
if ' Label' in df.columns: # Check if the original name with space exists
    df.rename(columns={' Label': 'Label'}, inplace=True)
elif 'Label' not in df.columns: # If not, it means it's already 'Label' or something else
    print("Warning: ' Label' column not found, assuming 'Label' is already correct or handled.")

# Print column names to diagnose the 'Label' issue (for debugging)
print("\nDataFrame Columns after stripping whitespace and renaming:")
print(df.columns.tolist())

# Replace infinite values with NaN, then remove rows with any missing values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()

# Drop columns with constant values (no variance)
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
df = df.drop(columns=constant_cols)

# Remove duplicated rows (if any)
df = df.drop_duplicates()

# Print column names just before accessing 'Label' for encoding (for debugging)
print("\nDataFrame Columns before encoding 'Label':")
print(df.columns.tolist())

# Encode 'Label' column: 'DDoS' as 1, 'BENIGN' as 0
df.loc[:, 'Label'] = df['Label'].apply(lambda x: 1 if x != 'BENIGN' else 0)

# Show cleaned dataset info
print("="*50)
print(" AFTER INITIAL CLEANING (Before Train-Test Split)")
print("="*50)
print(f"Total rows after cleaning: {len(df)}")
print("Label distribution after cleaning:")
print(df['Label'].value_counts())
print("="*50)
print(df.head()) # Using print instead of display for consistency

# ------------------------------------------------------------------------------

# STEP 3: Split data into training and testing sets FIRST

X = df.drop('Label', axis=1)
y = df['Label']

# Use stratify=y to maintain the class distribution in both train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\n" + "="*50)
print(" AFTER INITIAL TRAIN-TEST SPLIT")
print("="*50)
print(f"Training data shape (X_train): {X_train.shape}")
print(f"Testing data shape (X_test): {X_test.shape}")
print(f"Training label shape (y_train): {y_train.shape}")
print(f"Testing label shape (y_test): {y_test.shape}")
print("Training label distribution:")
print(y_train.value_counts())
print("Testing label distribution:")
print(y_test.value_counts())
print("="*50)

# ------------------------------------------------------------------------------

# STEP 4: Class Balancing with Undersampling (Applied ONLY to Training Data)

# Separate majority (DDoS) and minority (BENIGN) classes in the TRAINING SET
df_train = pd.concat([X_train, y_train], axis=1) # Recombine for easier filtering
df_train_majority = df_train[df_train['Label'] == 1] # DDoS in training
df_train_minority = df_train[df_train['Label'] == 0] # BENIGN in training

print("\nClass Distribution Before Balancing (Training Set):")
print(y_train.value_counts())

# Downsample the majority class (DDoS) in the training set
df_train_majority_downsampled = resample(df_train_majority,
                                         replace=False,
                                         n_samples=len(df_train_minority), # Match minority count in training
                                         random_state=42)

# Combine downsampled majority with minority class for the balanced training set
df_train_balanced = pd.concat([df_train_majority_downsampled, df_train_minority])

# Shuffle the combined training dataframe
df_train_balanced = df_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate X_train_balanced and y_train_balanced
X_train_balanced = df_train_balanced.drop('Label', axis=1)
y_train_balanced = df_train_balanced['Label']

# Show updated class balance for the training set
print("\n AFTER UNDERSAMPLING (Training Set Only)")
print("="*50)
print(f"Total rows in balanced training set: {len(df_train_balanced)}")
print("Class balance (Training Set):")
print(y_train_balanced.value_counts())
print("="*50)

# ------------------------------------------------------------------------------

# STEP 5: Display Sample Data (From Balanced Training Dataset)

# Columns to show in sample output
columns_to_show = ['Destination Port', 'Flow Duration', 'Label']

# Count label values (from balanced training data)
ddos_count_train = df_train_balanced[df_train_balanced['Label'] == 1].shape[0]
benign_count_train = df_train_balanced[df_train_balanced['Label'] == 0].shape[0]
total_rows_train = df_train_balanced.shape[0]

print("\n" + "="*60)
print("SAMPLE OF BALANCED TRAINING DATAFRAME")
print("="*60)
print(f"Total Rows       : {total_rows_train}")
print(f"DDoS Attacks     : {ddos_count_train}")
print(f"Benign Records   : {benign_count_train}")
print("="*60)

# Display sample rows
print("\n Balanced Training DataFrame (First 5 Rows):")
print(df_train_balanced[columns_to_show].head(5)) # Using print instead of display

print("\n DDoS Samples (Balanced Training Set - First 5):")
print(df_train_balanced[df_train_balanced['Label'] == 1][columns_to_show].head(5))

print("\n Benign Samples (Balanced Training Set - First 5):")
print(df_train_balanced[df_train_balanced['Label'] == 0][columns_to_show].head(5))

# ------------------------------------------------------------------------------

# STEP 6: Extract Statistical Features (Applied Separately to Train and Test Sets)

def extract_features(df_input): # Renamed parameter to avoid conflict with global df
    """Extracts statistical features from the dataframe."""
    # Create a trimmed preview column from flow characteristics
    df_input.loc[:, 'Flow Preview'] = df_input[['Destination Port', 'Flow Duration']].astype(str).agg(' | '.join, axis=1)

    # Feature 1: Packet length difference
    df_input.loc[:, 'pkt_length_diff'] = df_input['Max Packet Length'] - df_input['Min Packet Length']

    # Feature 2: Packet length variation (Max / Mean)
    df_input.loc[:, 'pkt_length_var_ratio'] = (df_input['Max Packet Length'] / (df_input['Packet Length Mean'] + 1e-5)).round(3)

    # Feature 3: Byte ratio (Fwd vs Bwd)
    df_input.loc[:, 'byte_ratio'] = (df_input['Total Length of Fwd Packets'] / (df_input['Total Length of Bwd Packets'] + 1e-5)).round(3)

    # Feature 4: Flow duration per packet
    df_input.loc[:, 'duration_per_packet'] = (df_input['Flow Duration'] / (df_input['Total Fwd Packets'] + df_input['Total Backward Packets'] + 1e-5)).round(3)

    # Feature 5: Average packet size vs Max packet size
    df_input.loc[:, 'avg_to_max_ratio'] = (df_input['Average Packet Size'] / (df_input['Max Packet Length'] + 1e-5)).round(3)

    # Remove duplicated columns (just in case, although should not happen after split)
    df_input = df_input.loc[:, ~df_input.columns.duplicated()]

    return df_input

# Apply feature extraction to the balanced training set and the original test set
X_train_featured = extract_features(X_train_balanced.copy()) # Use X_train_balanced here
X_test_featured = extract_features(X_test.copy()) # Use X_test here

# Drop the 'Flow Preview' column as it's a string and not suitable for ML models
X_train_featured = X_train_featured.drop(columns=['Flow Preview'])
X_test_featured = X_test_featured.drop(columns=['Flow Preview'])


# Preview extracted features from training set
print("\n Sample Extracted Features (Training Set - First 5):")
feature_cols_display = ['pkt_length_diff', 'pkt_length_var_ratio',
                        'byte_ratio', 'duration_per_packet', 'avg_to_max_ratio']
print(X_train_featured[feature_cols_display].head(5))

# Show final column list for training set
print("\n Final Columns (Training Set after Feature Extraction):")
print(X_train_featured.columns.tolist())

print("\n Sample Extracted Features (Testing Set - First 5):")
print(X_test_featured[feature_cols_display].head(5))

# Show final column list for testing set
print("\n Final Columns (Testing Set after Feature Extraction):")
print(X_test_featured.columns.tolist())


# ------------------------------------------------------------------------------

# STEP 7: Feature Scaling (Standardization)

# Initialize StandardScaler
scaler = StandardScaler()

# Identify numerical columns for scaling
numerical_cols = X_train_featured.select_dtypes(include=np.number).columns.tolist()

# Fit the scaler ONLY on the training data
scaler.fit(X_train_featured[numerical_cols])

# Transform both training and testing data
X_train_scaled = scaler.transform(X_train_featured[numerical_cols])
X_test_scaled = scaler.transform(X_test_featured[numerical_cols])

# Convert scaled arrays back to DataFrames, preserving column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=numerical_cols, index=X_train_featured.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=numerical_cols, index=X_test_featured.index)

print("\n" + "="*50)
print(" AFTER FEATURE SCALING")
print("="*50)
print(f"Training data shape (X_train_scaled): {X_train_scaled.shape}")
print(f"Testing data shape (X_test_scaled): {X_test_scaled.shape}")
print("\nSample of Scaled Training Data (First 5 Rows):")
print(X_train_scaled.head(5))
print("\nSample of Scaled Testing Data (First 5 Rows):")
print(X_test_scaled.head(5))
print("="*50)

# The preprocessed data is now ready for model training: X_train_scaled, y_train_balanced, X_test_scaled, y_test
# Note: y_train is now y_train_balanced


 INITIAL DATASET INFO
Total rows before cleaning: 225745
Initial Label distribution:
 Label
DDoS      128027
BENIGN     97718
Name: count, dtype: int64

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225745 entries, 0 to 225744
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             225745 non-null  int64  
 1    Flow Duration                225745 non-null  int64  
 2    Total Fwd Packets            225745 non-null  int64  
 3    Total Backward Packets       225745 non-null  int64  
 4   Total Length of Fwd Packets   225745 non-null  int64  
 5    Total Length of Bwd Packets  225745 non-null  int64  
 6    Fwd Packet Length Max        225745 non-null  int64  
 7    Fwd Packet Length Min        225745 non-null  int64  
 8    Fwd Packet Length Mean       225745 non-null  float64
 9    Fwd Packet Length Std        225745 non-null  float64


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,225745.00000,2.257450e+05,225745.000000,225745.000000,225745.000000,2.257450e+05,225745.000000,225745.000000,225745.000000,225745.000000,...,225745.000000,225745.000000,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05
mean,8879.61946,1.624165e+07,4.874916,4.572775,939.463346,5.960477e+03,538.535693,27.882221,164.826715,214.907242,...,3.311497,21.482753,1.848261e+05,1.293436e+04,2.080849e+05,1.776201e+05,1.032214e+07,3.611943e+06,1.287813e+07,7.755355e+06
std,19754.64740,3.152437e+07,15.422874,21.755356,3249.403484,3.921834e+04,1864.128991,163.324159,504.892965,797.411073,...,12.270018,4.166799,7.979250e+05,2.102737e+05,9.002350e+05,7.842602e+05,2.185303e+07,1.275689e+07,2.692126e+07,1.983109e+07
min,0.00000,-1.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,80.00000,7.118000e+04,2.000000,1.000000,26.000000,0.000000e+00,6.000000,0.000000,6.000000,0.000000,...,1.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,80.00000,1.452333e+06,3.000000,4.000000,30.000000,1.640000e+02,20.000000,0.000000,8.666667,5.301991,...,2.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,80.00000,8.805237e+06,5.000000,5.000000,63.000000,1.160100e+04,34.000000,6.000000,32.000000,10.263203,...,4.000000,20.000000,1.878000e+03,0.000000e+00,1.878000e+03,1.862000e+03,8.239725e+06,0.000000e+00,8.253838e+06,7.422849e+06
max,65532.00000,1.199999e+08,1932.000000,2942.000000,183012.000000,5.172346e+06,11680.000000,1472.000000,3867.000000,6692.644993,...,1931.000000,52.000000,1.000000e+08,3.950000e+07,1.000000e+08,1.000000e+08,1.200000e+08,6.530000e+07,1.200000e+08,1.200000e+08



DataFrame Columns after stripping whitespace and renaming:
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0



Class Distribution Before Balancing:
Label
1    128014
0     95068
Name: count, dtype: int64

 AFTER UNDERSAMPLING
Total rows after balancing: 190136
Class balance:
Label
1    95068
0    95068
Name: count, dtype: int64

SAMPLE OF BALANCED DATAFRAME
Total Rows       : 190136
DDoS Attacks     : 95068
Benign Records   : 95068

 Balanced DataFrame (First 5 Rows):


...

,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
6233,0,1.000,2400000.000,1147984.380,1.250
139480,5840,5.023,0.002,200535.333,0.221
35733,1398,8.814,0.211,3653563.765,0.117
181334,0,1.000,1800000.000,132168.893,1.333
137899,0,1.000,2400000.000,5453592.866,1.250



 Final Columns (Testing Set after Feature Extraction):
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'ECE Flag Count', 'Down/Up R

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
179040,-0.342657,3.182493,1.006419,0.337656,0.533107,0.012541,0.330052,-0.182382,-0.108150,0.143690,...,-0.206558,4.930281,-0.278615,3.915530,5.555942,-0.028324,2.157676,-0.042792,0.861048,-1.087555
176170,-0.493664,2.296464,0.177937,0.046861,-0.301709,0.136376,-0.306709,-0.182382,-0.343985,-0.288823,...,-0.230019,1.575320,3.450283,2.433455,0.174253,0.753826,1.068635,-0.042792,2.157719,-0.983046
136164,-0.493664,-0.504360,-0.117950,0.046861,-0.310257,0.136376,-0.306709,-0.182382,-0.340932,-0.283491,...,-0.231015,-0.469038,-0.278615,-0.474413,-0.389833,0.753826,0.338006,-0.042792,-0.564115,-0.823209
90783,-0.493664,3.140114,0.888065,0.503825,-0.117651,-0.088607,-0.142687,-0.182382,-0.292505,-0.170312,...,-0.182955,0.001672,-0.274671,-0.091700,0.118353,-0.534453,2.951175,-0.042792,0.767622,-1.124440
136319,-0.494943,-0.512896,-0.058773,-0.119309,-0.265240,-0.132307,-0.293825,0.076654,-0.272535,-0.295371,...,-0.231015,-0.469038,-0.278615,-0.474413,-0.389833,-0.788852,-0.641162,-0.042792,-0.576310,0.141962



Sample of Scaled Testing Data (First 5 Rows):


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
6233,-0.493664,-0.369329,-0.058773,-0.202393,-0.310827,-0.137353,-0.313646,-0.148595,-0.345817,-0.295371,...,-0.231015,-0.469038,-0.278615,-0.474413,-0.389833,-0.805136,-0.842989,0.041817,-0.085974,1.285413
139480,-0.493664,-0.457057,-0.117950,0.046861,-0.310257,0.136235,-0.306709,-0.182382,-0.340932,-0.283491,...,-0.231015,-0.469038,-0.278615,-0.474413,-0.389833,0.753826,0.338887,-0.042792,-0.492472,-0.823209
35733,-0.476469,3.166121,0.769710,0.379199,-0.057818,-0.035427,-0.166968,-0.182382,-0.263986,-0.194301,...,-0.185353,2.212162,-0.269792,1.709302,2.562727,-0.431946,1.452605,-0.042792,0.989033,-1.036325
181334,-0.476469,-0.501385,-0.117950,-0.202393,-0.312536,-0.137353,-0.313646,-0.148595,-0.345817,-0.295371,...,-0.231015,-0.469038,-0.278615,-0.474413,-0.389833,-0.805136,-0.842989,0.020665,-0.521804,1.455497
137899,-0.493664,0.172765,-0.058773,-0.202393,-0.310827,-0.137353,-0.313646,-0.148595,-0.345817,-0.295371,...,-0.229707,0.536989,-0.278615,0.343542,0.718012,-0.805136,-0.842989,0.041817,1.761327,1.285413
